In [1]:
import pandas as pd
import numpy as np
import tweepy

# Data collection

Here I get tweets based on their id using `tweepy` module. Then remove the tweets which were deleted. Then thos data that their tweets were removed and save new datasets as `csv` in data folder.

In [2]:
TRAIN_DATA_DIR = "data/isarcasm_train.csv"
TEST_DATA_DIR = "data/isarcasm_test.csv"

In [3]:
train_df = pd.read_csv(TRAIN_DATA_DIR)
test_df = pd.read_csv(TEST_DATA_DIR)

In [4]:
train_df.head()

,tweet_id,sarcasm_label,sarcasm_type
0,992251158159765504,not_sarcastic,NaN
1,1084373658745876480,not_sarcastic,NaN
2,812181186,not_sarcastic,NaN
3,1090276741329928194,not_sarcastic,NaN
4,553070692682723329,sarcastic,sarcasm


In [5]:
test_df.head()

,tweet_id,sarcasm_label,sarcasm_type
0,933721764970057729,sarcastic,sarcasm
1,1086119014131208193,not_sarcastic,NaN
2,657671293756567552,not_sarcastic,NaN
3,914242426755256320,not_sarcastic,NaN
4,1088604537211047936,sarcastic,satire


In [6]:
consumer_key = 'XX'
consumer_key_secret = 'XX'
access_token = 'XX'
access_token_secret = 'XX'

In [8]:
auth = tweepy.OAuthHandler(consumer_key, consumer_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [9]:
def get_tweet(tweet_id):
    try:
        tweet_fetched = api.get_status(str(tweet_id))
        return tweet_fetched.text
    except:
        return None

In [16]:
tweets_body = train_df.apply(lambda row: get_tweet(row['tweet_id']), axis=1)

In [20]:
train_df['tweet_body'] = tweets_body

In [24]:
train_df.info

<bound method DataFrame.info of                  tweet_id  sarcasm_label sarcasm_type  \
0      992251158159765504  not_sarcastic          NaN   
1     1084373658745876480  not_sarcastic          NaN   
2               812181186  not_sarcastic          NaN   
3     1090276741329928194  not_sarcastic          NaN   
4      553070692682723329      sarcastic      sarcasm   
...                   ...            ...          ...   
3572   880621198811508737  not_sarcastic          NaN   
3573   977938081805414400  not_sarcastic          NaN   
3574  1145291654926147586      sarcastic        irony   
3575  1146574582272462853  not_sarcastic          NaN   
3576  1055962756380327937  not_sarcastic          NaN   

                                             tweet_body  
0                                                  None  
1     sorry but sue from the mediweight advert looks...  
2     @davesnyder since this is fantasy can it be an...  
3                                                  

In [26]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3577 entries, 0 to 3576
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   tweet_id       3577 non-null   int64 
 1   sarcasm_label  3577 non-null   object
 2   sarcasm_type   619 non-null    object
 3   tweet_body     2859 non-null   object
dtypes: int64(1), object(3)
memory usage: 111.9+ KB


In [28]:
train_df = train_df[train_df['tweet_body'].notna()]

In [29]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2859 entries, 1 to 3576
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   tweet_id       2859 non-null   int64 
 1   sarcasm_label  2859 non-null   object
 2   sarcasm_type   488 non-null    object
 3   tweet_body     2859 non-null   object
dtypes: int64(1), object(3)
memory usage: 111.7+ KB


In [31]:
tweets_body = test_df.apply(lambda row: get_tweet(row['tweet_id']), axis=1)

In [34]:
test_df['tweet_body'] = tweets_body 
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 907 entries, 0 to 906
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   tweet_id       907 non-null    int64 
 1   sarcasm_label  907 non-null    object
 2   sarcasm_type   158 non-null    object
 3   tweet_body     714 non-null    object
dtypes: int64(1), object(3)
memory usage: 28.5+ KB


In [35]:
test_df = test_df[test_df['tweet_body'].notna()]

In [36]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 1 to 906
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   tweet_id       714 non-null    int64 
 1   sarcasm_label  714 non-null    object
 2   sarcasm_type   121 non-null    object
 3   tweet_body     714 non-null    object
dtypes: int64(1), object(3)
memory usage: 27.9+ KB


In [37]:
train_df.to_csv(r'data/train.csv', index = False, header = True)
test_df.to_csv(r'data/test.csv', index = False, header = True)

In [39]:
max_len = 0
for i in range(len(train_df)):
    leng = len(train_df.iloc[i]["tweet_body"])
    if leng > max_len:
        max_len = leng
print(leng)

140


In [147]:
def extract_hashtags(text):
    hashtag_list = []
      
    for word in text.split():
        if word[0] == '#' and len(word) > 1 and not word[1].isdigit():
            hashtag_list.append(word[1:].split('.')[0].split('…')[0])
    
    return hashtag_list

In [43]:
from tqdm import tqdm

tqdm.pandas()

In [148]:
test_df = pd.read_csv('data/test.csv')
train_df = pd.read_csv('data/train.csv')

In [149]:
hashtags_train = train_df.progress_apply(lambda row: extract_hashtags(row['tweet_body']), axis=1)
hashtags_test = test_df.progress_apply(lambda row: extract_hashtags(row['tweet_body']), axis=1)
train_df['hashtags'] = hashtags_train
test_df['hashtags'] = hashtags_test

100%|██████████████████████████████████████| 714/714 [00:00<00:00, 49413.14it/s]


In [153]:
import regex as re

emoji = re.compile('[\\u203C-\\u3299\\U0001F000-\\U0001F644]')

def extract_emojies(text):
    return re.findall(emoji, text)

In [154]:
emojies_train = train_df.progress_apply(lambda row: extract_emojies(row['tweet_body']), axis=1)
emojies_test = test_df.progress_apply(lambda row: extract_emojies(row['tweet_body']), axis=1)
train_df['emojies'] = emojies_train
test_df['emojies'] = emojies_test

100%|██████████████████████████████████████| 714/714 [00:00<00:00, 29859.25it/s]


In [158]:
import re

def remove_links(text):
    return re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text)

In [159]:
tweets_bodies_without_links_train = train_df.progress_apply(lambda row: remove_links(row['tweet_body']), axis=1)
tweets_bodies_without_links_test = test_df.progress_apply(lambda row: remove_links(row['tweet_body']), axis=1)
train_df['tweet_body'] = tweets_bodies_without_links_train
test_df['tweet_body'] = tweets_bodies_without_links_test

100%|██████████████████████████████████████| 714/714 [00:00<00:00, 48704.35it/s]


In [161]:
train_df

,tweet_id,sarcasm_label,sarcasm_type,tweet_body,hashtags,emojies
0,1084373658745876480,not_sarcastic,NaN,sorry but sue from the mediweight advert looks...,[],[]
1,812181186,not_sarcastic,NaN,@davesnyder since this is fantasy can it be an...,[],[]
2,1090351571395899392,sarcastic,irony,"Good times, fun times here in Grand old Britan...",[],[]
3,914961092698353664,not_sarcastic,NaN,I send light and love to all impacted by the #...,"[vegasshooting, VegasStrong]",[]
4,1080104089277681664,not_sarcastic,NaN,Another year closer to death,[],[]
...,...,...,...,...,...,...
2854,1131854999246524416,not_sarcastic,NaN,why did i think “knackered” was a swear word w...,[],[]
2855,1171093461111660546,not_sarcastic,NaN,where are the best places to get an academic d...,[],[☹]
2856,1158305504575275009,not_sarcastic,NaN,i accidentally just posted someone’s happy bir...,[],[]
2857,1145291654926147586,sarcastic,irony,I can confirm I am retiring from the consumpti...,[],[]


In [164]:
train_df['sarcasm_label'].replace({'sarcastic': 1, 'not_sarcastic': 0}, inplace=True)
test_df['sarcasm_label'].replace({'sarcastic': 1, 'not_sarcastic': 0}, inplace=True)

In [167]:
train_df['sarcasm_label'].value_counts()

0    2371
1     488
Name: sarcasm_label, dtype: int64

As we can see it's not balanced. This can be a #problem.

For this, we will compare the ratio with <a href="https://aclanthology.org/P19-1239/">Cai et al.(2019) papaer</a>.

In that paper we can see that the ration in train dataset is 8642/11174 which is nearly 0.77.

Here but the ration is 488/2371 which is nearly 0.2.

There is a huge difference.

## Model using BERT(ReBERTa)

### Setting the hyper parameters

|Parameter  	    |Ours  	    
|-	                |-	        
|Epochs  	        |1  	    
|Batch Size  	    |32  	    
|Seq Length  	    |75  	    
|Learning Rate      |5e-5       
|Weight decay       |1e-2       
|Warmup rate        |0.2        
|Gradient Clipping  |1.0        

Absoultely it is out start point and we try to make it better in the future.

In [168]:
train_args = {
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'sliding_window': False,
    'max_seq_length': 75,
    'learning_rate': 0.00005,
    'weight_decay': 0.01,
    'warmup_ratio': 0.2,
    'max_grad_norm': 1.0,
    'num_train_epochs': 1,
    'train_batch_size': 32,
    'save_model_every_epoch': False,
    'save_steps': 4000,
    'fp16': True,
    'output_dir': '/outputs/',
    'evaluate_during_training': True,
}

As these days transformers are so popular we will use <a href="https://huggingface.co/roberta-base">roberta-base</a> model by hugging face.

<img src="https://huggingface.co/front/assets/huggingface_logo.svg">

#### What is ReBERTa base model?
Pretrained model on English language using a masked language modeling (MLM) objective.  It was introduced in <a href="https://arxiv.org/abs/1907.11692">this paper</a> and first released in <a href="https://github.com/pytorch/fairseq/tree/master/examples/roberta">this repository</a>. This model is case-sensitive: it makes a difference between english and English.